<a href="https://colab.research.google.com/github/OmegaTrees/JupiterLab-Render/blob/main/Google%20collab%20renamer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:

# Install nest_asyncio to handle Colab's event loop
!pip install nest-asyncio

In [3]:

!pip install pyrogram tgcrypto

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 4.4 MB/s eta 0:00:00
  Created wheel for pyaes: filename=pyaes-1.6.1-py3-none-any.whl size=26347 sha256=c1325ab0a9ad58fa5926111052a789965ce4acaa3155d2bd4a356a86da26f606
  Stored in directory: /root/.cache/pip/wheels/4e/52/33/010d0843550bffb6a591b11629070ae140c0ad4f53e68a3bd3
Successfully built pyaes


In [4]:

%%writefile progress_tracker.py
import time
import asyncio
import threading
import os
from pyrogram.types import Message
from typing import Optional
from concurrent.futures import ThreadPoolExecutor

class ProgressTracker:
    def __init__(self, message: Message, filename: str, operation: str = "Processing"):
        self.message = message
        self.filename = filename
        self.operation = operation
        self.last_update = 0
        self.start_time = time.time()
        self.last_percentage = 0
        self.loop = None

    async def update(self, current: int, total: int):
        """Async progress update"""
        current_time = time.time()

        # Calculate progress
        percentage = (current / total) * 100 if total > 0 else 0

        # Update every 2 seconds OR every 5% progress to avoid rate limits
        if (current_time - self.last_update < 2) and (percentage - self.last_percentage < 5):
            return

        self.last_update = current_time
        self.last_percentage = percentage

        current_mb = current / (1024 * 1024)
        total_mb = total / (1024 * 1024)

        # Calculate speed
        elapsed_time = current_time - self.start_time
        if elapsed_time > 0:
            speed = current / elapsed_time
            speed_mb = speed / (1024 * 1024)
        else:
            speed_mb = 0

        # Calculate ETA
        if speed > 0:
            remaining_bytes = total - current
            eta_seconds = remaining_bytes / speed
            eta_str = self.format_time(eta_seconds)
        else:
            eta_str = "Unknown"

        # Create progress bar
        progress_bar = self.create_progress_bar(percentage)

        # Choose emoji based on operation
        emoji = "⬇️" if "download" in self.operation.lower() else "⬆️"

        progress_text = (
            f"{emoji} **{self.operation}:** `{self.filename}`\n\n"
            f"{progress_bar} {percentage:.1f}%\n\n"
            f"📊 **Size:** {current_mb:.1f} / {total_mb:.1f} MB\n"
            f"🚀 **Speed:** {speed_mb:.1f} MB/s\n"
            f"⏰ **ETA:** {eta_str}"
        )

        try:
            await self.message.edit_text(progress_text)
        except Exception as e:
            pass  # Ignore telegram rate limit errors

    def create_progress_bar(self, percentage: float, length: int = 20) -> str:
        """Create a visual progress bar"""
        filled = int(length * percentage / 100)
        bar = "█" * filled + "▒" * (length - filled)
        return f"[{bar}]"

    def format_time(self, seconds: float) -> str:
        """Format time in human readable format"""
        if seconds < 60:
            return f"{int(seconds)}s"
        elif seconds < 3600:
            return f"{int(seconds // 60)}m {int(seconds % 60)}s"
        else:
            hours = int(seconds // 3600)
            minutes = int((seconds % 3600) // 60)
            return f"{hours}h {minutes}m"

class BatchProgressTracker:
    """Progress tracker for batch operations (Mode 3)"""
    def __init__(self, message: Message, total_files: int):
        self.message = message
        self.total_files = total_files
        self.current_file = 0
        self.successful = 0
        self.failed = 0
        self.current_filename = ""
        self.current_operation = ""
        self.start_time = time.time()

    async def update_file(self, file_index: int, filename: str, operation: str = "Processing"):
        """Update current file being processed"""
        self.current_file = file_index + 1
        self.current_filename = filename
        self.current_operation = operation
        await self.update_display()

    async def mark_success(self):
        """Mark current file as successful"""
        self.successful += 1
        await self.update_display()

    async def mark_failed(self):
        """Mark current file as failed"""
        self.failed += 1
        await self.update_display()

    async def update_display(self):
        """Update the progress display"""
        percentage = (self.current_file / self.total_files) * 100 if self.total_files > 0 else 0
        progress_bar = self.create_progress_bar(percentage)

        elapsed_time = time.time() - self.start_time
        elapsed_str = self.format_time(elapsed_time)

        # Calculate ETA
        if self.current_file > 0:
            avg_time_per_file = elapsed_time / self.current_file
            remaining_files = self.total_files - self.current_file
            eta_seconds = remaining_files * avg_time_per_file
            eta_str = self.format_time(eta_seconds)
        else:
            eta_str = "Unknown"

        progress_text = (
            f"🔄 **Batch Processing**\n\n"
            f"{progress_bar} {percentage:.1f}%\n\n"
            f"📊 **Progress:** {self.current_file}/{self.total_files}\n"
            f"📁 **Current:** `{self.current_filename}`\n"
            f"🔧 **Operation:** {self.current_operation}\n\n"
            f"✅ **Successful:** {self.successful}\n"
            f"❌ **Failed:** {self.failed}\n\n"
            f"⏱️ **Elapsed:** {elapsed_str}\n"
            f"⏰ **ETA:** {eta_str}"
        )

        try:
            await self.message.edit_text(progress_text)
        except Exception:
            pass

    def create_progress_bar(self, percentage: float, length: int = 20) -> str:
        """Create a visual progress bar"""
        filled = int(length * percentage / 100)
        bar = "█" * filled + "▒" * (length - filled)
        return f"[{bar}]"

    def format_time(self, seconds: float) -> str:
        """Format time in human readable format"""
        if seconds < 60:
            return f"{int(seconds)}s"
        elif seconds < 3600:
            return f"{int(seconds // 60)}m {int(seconds % 60)}s"
        else:
            hours = int(seconds // 3600)
            minutes = int((seconds % 3600) // 60)
            return f"{hours}h {minutes}m"

# Thread-safe progress handler
class ThreadSafeProgressHandler:
    def __init__(self, message: Message, filename: str, operation: str, known_file_size: int = 0):
        self.message = message
        self.filename = filename
        self.operation = operation
        self.known_file_size = known_file_size  # Pre-known file size in bytes
        self.last_update = 0
        self.start_time = time.time()
        self.executor = ThreadPoolExecutor(max_workers=1)
        self.total_size = known_file_size  # Start with known size
        print(f"DEBUG: ThreadSafeProgressHandler initialized with known_file_size = {known_file_size} bytes ({known_file_size/(1024*1024):.1f} MB)")

    def progress_callback(self, current: int, total: int):
        """Thread-safe progress callback"""
        current_time = time.time()

        print(f"DEBUG: Pyrogram callback - current: {current}, total: {total}, known_size: {self.known_file_size}")

        # Force use known file size for downloads - Pyrogram download progress is unreliable
        if self.known_file_size > 0:
            total = self.known_file_size

        # Store the correct total
        if total > 0:
            self.total_size = total

        # Only update every 2 seconds to avoid rate limits
        if current_time - self.last_update < 2:
            return

        self.last_update = current_time

        # Schedule the update in a thread-safe way
        self.executor.submit(self._sync_update, current, total)

    def _sync_update(self, current: int, total: int):
        """Synchronous update that creates new event loop"""
        try:
            # Create new event loop for this thread
            loop = asyncio.new_event_loop()
            asyncio.set_event_loop(loop)

            # Run the async update
            loop.run_until_complete(self._async_update(current, total))

        except Exception as e:
            print(f"Progress update error: {e}")
        finally:
            try:
                loop.close()
            except:
                pass

    async def _async_update(self, current: int, total: int):
        """Async update method"""
        current_time = time.time()

        # Ensure we don't exceed total
        if current > total and total > 0:
            current = total

        # Calculate progress
        percentage = (current / total) * 100 if total > 0 else 0
        current_mb = current / (1024 * 1024)
        total_mb = total / (1024 * 1024)

        # Calculate speed
        elapsed_time = current_time - self.start_time
        if elapsed_time > 0:
            speed = current / elapsed_time
            speed_mb = speed / (1024 * 1024)
        else:
            speed_mb = 0

        # Calculate ETA
        if speed > 0 and current < total:
            remaining_bytes = total - current
            eta_seconds = remaining_bytes / speed
            eta_str = self._format_time(eta_seconds)
        else:
            eta_str = "Complete" if current >= total else "Unknown"

        # Create progress bar
        filled = int(20 * percentage / 100)
        bar = "█" * filled + "▒" * (20 - filled)
        progress_bar = f"[{bar}]"

        # Choose emoji based on operation
        emoji = "⬇️" if "download" in self.operation.lower() else "⬆️"

        progress_text = (
            f"{emoji} **{self.operation}:** `{self.filename}`\n\n"
            f"{progress_bar} {percentage:.1f}%\n\n"
            f"📊 **Size:** {current_mb:.1f} / {total_mb:.1f} MB\n"
            f"🚀 **Speed:** {speed_mb:.1f} MB/s\n"
            f"⏰ **ETA:** {eta_str}"
        )

        try:
            await self.message.edit_text(progress_text)
        except Exception:
            pass  # Ignore telegram errors

    def _format_time(self, seconds: float) -> str:
        """Format time in human readable format"""
        if seconds < 0 or seconds > 86400:  # More than 24 hours
            return "Unknown"
        elif seconds < 60:
            return f"{int(seconds)}s"
        elif seconds < 3600:
            return f"{int(seconds // 60)}m {int(seconds % 60)}s"
        else:
            hours = int(seconds // 3600)
            minutes = int((seconds % 3600) // 60)
            return f"{hours}h {minutes}m"

    def cleanup(self):
        """Clean up the executor"""
        try:
            self.executor.shutdown(wait=False)
        except:
            pass

async def download_with_progress(client, file_id: str, file_path: str, progress_msg: Message, filename: str, file_size: int = 0):
    """Helper function to download file with progress tracking"""

    print(f"DEBUG: download_with_progress called with file_size = {file_size} bytes ({file_size/(1024*1024):.1f} MB)")

    # Initial message
    await progress_msg.edit_text(
        f"⬇️ **Downloading:** `{filename}`\n\n"
        f"⏳ **Starting download...**"
    )

    # Create thread-safe progress handler with known file size
    handler = ThreadSafeProgressHandler(progress_msg, filename, "Downloading", file_size)

    try:
        # Download with progress
        await client.download_media(
            file_id,
            file_name=file_path,
            progress=handler.progress_callback
        )

        # Final success message
        await progress_msg.edit_text(
            f"✅ **Download Complete:** `{filename}`"
        )

    except Exception as e:
        await progress_msg.edit_text(f"❌ **Download Failed:** {str(e)}")
        raise e
    finally:
        handler.cleanup()

async def upload_with_progress(client, chat_id: int, file_path: str, progress_msg: Message,
                             filename: str, caption: str = None, thumb: str = None):
    """Helper function to upload file with progress tracking"""

    # Get file size for upload progress
    upload_file_size = 0
    if os.path.exists(file_path):
        upload_file_size = os.path.getsize(file_path)

    print(f"DEBUG: upload_with_progress called with file_size = {upload_file_size} bytes ({upload_file_size/(1024*1024):.1f} MB)")

    # Initial message
    await progress_msg.edit_text(
        f"⬆️ **Uploading:** `{filename}`\n\n"
        f"⏳ **Starting upload...**"
    )

    # Create thread-safe progress handler with known file size
    handler = ThreadSafeProgressHandler(progress_msg, filename, "Uploading", upload_file_size)

    try:
        # Upload with progress
        result = await client.send_document(
            chat_id=chat_id,
            document=file_path,
            caption=caption,
            thumb=thumb,
            progress=handler.progress_callback
        )

        # Final success message
        await progress_msg.edit_text(
            f"✅ **Upload Complete:** `{filename}`"
        )

        return result

    except Exception as e:
        await progress_msg.edit_text(f"❌ **Upload Failed:** {str(e)}")
        raise e
    finally:
        handler.cleanup()

Writing progress_tracker.py


In [5]:

import getpass

# Prompt for bot credentials
API_ID = input("🔢 Enter your API ID: ")
API_HASH = getpass.getpass("🔐 Enter your API HASH (input hidden): ")
BOT_TOKEN = getpass.getpass("🤖 Enter your Bot Token (input hidden): ")

print("\n✅ Configuration set!")
print("📝 Your credentials have been entered successfully.")

🔢 Enter your API ID: 25217592
🔐 Enter your API HASH (input hidden): ··········
🤖 Enter your Bot Token (input hidden): ··········

✅ Configuration set!
📝 Your credentials have been entered successfully.


In [6]:

%%writefile video_rename_bot.py
import os
import re
import asyncio
import logging
import json
import shutil
from typing import Dict, List, Optional
from pathlib import Path
from pyrogram import Client, filters
from pyrogram.types import Message, InlineKeyboardMarkup, InlineKeyboardButton
from datetime import datetime

# Import progress tracker
from progress_tracker import ProgressTracker, BatchProgressTracker, download_with_progress, upload_with_progress

# Set up logging for Colab
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Supported video formats
VIDEO_FORMATS = {'.mkv', '.mp4', '.avi', '.mov', '.wmv', '.flv', '.webm', '.m4v', '.3gp', '.ts', '.mts', '.m2ts'}

# Create downloads directory
DOWNLOAD_DIR = "/content/downloads"
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

class VideoRenameBot:
    def __init__(self):
        self.user_states = {}
        self.settings = {}
        self.load_settings()

    def load_settings(self):
        """Load bot settings from file"""
        try:
            if os.path.exists("/content/bot_settings.json"):
                with open("/content/bot_settings.json", "r") as f:
                    self.settings = json.load(f)
            else:
                self.settings = {
                    "max_downloads": 5,
                    "mode2_format": "[{episode}] {name} {resolution}.{ext}",
                    "preserve_extension": True,
                    "caption_mode": "custom"
                }
                self.save_settings()
        except Exception as e:
            logger.error(f"Error loading settings: {e}")
            self.settings = {
                "max_downloads": 5,
                "mode2_format": "[{episode}] {name} {resolution}.{ext}",
                "preserve_extension": True,
                "caption_mode": "custom"
            }

    def save_settings(self):
        """Save bot settings to file"""
        try:
            with open("/content/bot_settings.json", "w") as f:
                json.dump(self.settings, f, indent=2)
        except Exception as e:
            logger.error(f"Error saving settings: {e}")

    def get_user_state(self, user_id: int) -> Dict:
        """Get user state"""
        if user_id not in self.user_states:
            self.user_states[user_id] = {
                "mode": None,
                "thumbnail": None,
                "rename_list": [],
                "current_rename_index": 0,
                "current_episode": 1,
                "waiting_for_list": False,
                "waiting_for_thumbnail": False,
                "waiting_for_format": False,
                "waiting_for_video": False,
                "waiting_for_url_range": False,
                "processing": False,
                "url_range": {},
                "temp_video": None
            }
        return self.user_states[user_id]

    def reset_user_state(self, user_id: int):
        """Reset user state"""
        if user_id in self.user_states:
            self.user_states[user_id] = {
                "mode": None,
                "thumbnail": None,
                "rename_list": [],
                "current_rename_index": 0,
                "current_episode": 1,
                "waiting_for_list": False,
                "waiting_for_thumbnail": False,
                "waiting_for_format": False,
                "waiting_for_video": False,
                "waiting_for_url_range": False,
                "processing": False,
                "url_range": {},
                "temp_video": None
            }

    def is_video_file(self, filename: str) -> bool:
        """Check if file is a supported video format"""
        return Path(filename).suffix.lower() in VIDEO_FORMATS

    def extract_metadata(self, filename: str) -> Dict:
        """Extract metadata from filename"""
        metadata = {
            "name": "",
            "episode": "",
            "season": "",
            "resolution": "",
            "quality": ""
        }

        try:
            name_without_ext = Path(filename).stem

            # Extract resolution
            resolution_patterns = [
                r'(\d{3,4}[pP])',
                r'(720p|1080p|1440p|2160p|4K|8K)',
                r'(HD|FHD|UHD)'
            ]

            for pattern in resolution_patterns:
                match = re.search(pattern, name_without_ext, re.IGNORECASE)
                if match:
                    metadata["resolution"] = match.group(1)
                    break

            # Extract episode
            episode_patterns = [
                r'[Ee](\d+)',
                r'[Ee]pisode[\s\.]?(\d+)',
                r'S\d+E(\d+)',
                r'[\s\-\.](\d{1,3})[\s\-\.]'
            ]

            for pattern in episode_patterns:
                match = re.search(pattern, name_without_ext)
                if match:
                    metadata["episode"] = match.group(1).zfill(2)
                    break

            # Clean name
            clean_name = re.sub(r'\b(19|20)\d{2}\b', '', name_without_ext)
            clean_name = re.sub(r'\b\d+[pP]\b', '', clean_name)
            clean_name = re.sub(r'\b[Ss]\d+[Ee]\d+\b', '', clean_name)
            clean_name = re.sub(r'\b[Ee]pisode?\s?\d+\b', '', clean_name, flags=re.IGNORECASE)
            clean_name = re.sub(r'\b(BluRay|BDRip|WEBRip|WEB-DL|HDTV|x264|x265|HEVC|AVC|VIKI)\b', '', clean_name, flags=re.IGNORECASE)
            clean_name = re.sub(r'[\[\]\(\){}@_]', '', clean_name)
            clean_name = re.sub(r'[_\-\.]+', ' ', clean_name)
            clean_name = re.sub(r'\s+', ' ', clean_name).strip()

            metadata["name"] = clean_name if clean_name else "Unknown"

        except Exception as e:
            logger.error(f"Error extracting metadata: {e}")
            metadata["name"] = Path(filename).stem

        return metadata

    def format_filename_mode2(self, metadata: Dict, episode_num: int = None) -> str:
        """Format filename using Mode2 template"""
        try:
            format_template = self.settings.get("mode2_format", "[{episode}] {name} {resolution}.{ext}")

            episode = str(episode_num).zfill(2) if episode_num else metadata.get("episode", "01")

            format_vars = {
                "name": metadata.get("name", "Unknown"),
                "episode": episode,
                "season": metadata.get("season", "01"),
                "resolution": metadata.get("resolution", ""),
                "quality": metadata.get("quality", ""),
                "ext": "mp4"
            }

            formatted_name = format_template.format(**format_vars)
            formatted_name = re.sub(r'\s+', ' ', formatted_name)
            formatted_name = re.sub(r'\[\s*\]', '', formatted_name)
            formatted_name = re.sub(r'\(\s*\)', '', formatted_name)

            return formatted_name.strip()

        except Exception as e:
            logger.error(f"Error formatting filename: {e}")
            return f"Episode_{episode_num or 1}.mp4"

def create_bot_app(api_id, api_hash, bot_token):
    """Create and return bot application"""
    return Client("video_rename_bot", api_id=api_id, api_hash=api_hash, bot_token=bot_token, workdir="/content")

def setup_handlers(app, bot_instance):
    """Setup all bot handlers"""

    @app.on_message(filters.command("start"))
    async def start_command(client, message: Message):
        """Handle /start command"""
        try:
            start_text = (
                "🎬 **Video Rename Bot (Google Colab)**\n\n"
                "**Supported Formats:**\n"
                ".mkv, .mp4, .avi, .mov, .wmv, .flv, .webm, .m4v, .3gp, .ts, .mts, .m2ts\n\n"
                "**Available Commands:**\n"
                "• `/mode1` - Single video rename\n"
                "• `/mode2` - Auto rename from metadata\n"
                "• `/mode3` - URL range batch rename\n"
                "• `/thumbnail` - Set thumbnail\n"
                "• `/format` - Set Mode2 format\n"
                "• `/status` - Show bot status\n"
                "• `/reset` - Reset bot state\n\n"
                f"⚙️ **Max Downloads:** {bot_instance.settings.get('max_downloads', 5)}\n"
                f"📝 **Mode2 Format:** `{bot_instance.settings.get('mode2_format')}`\n"
                f"🔧 **Preserve Extension:** {'✅ Yes' if bot_instance.settings.get('preserve_extension', True) else '❌ No'}\n"
                f"💬 **Caption Mode:** {'📄 Original' if bot_instance.settings.get('caption_mode') == 'original' else '🔄 Custom'}\n\n"
                "⚠️ **Note:** Running on Google Colab - Files are downloaded, renamed, and re-uploaded"
            )

            await message.reply_text(start_text)

        except Exception as e:
            await message.reply_text(f"❌ Error: {str(e)}")

    @app.on_message(filters.command("mode1"))
    async def mode1_command(client, message: Message):
        """Handle /mode1 command - Simple one video, one name"""
        try:
            user_id = message.from_user.id
            user_state = bot_instance.get_user_state(user_id)

            if user_state.get("processing"):
                await message.reply_text("⏳ **Please wait for current process to finish!**")
                return

            user_state["mode"] = "mode1"
            user_state["temp_video"] = None
            user_state["waiting_for_video"] = True

            await message.reply_text(
                "📝 **Mode 1: Simple Rename**\n\n"
                "**How it works:**\n"
                "1️⃣ Send me ONE video file\n"
                "2️⃣ Send me ONE new name\n"
                "3️⃣ I'll download, rename, and re-upload\n\n"
                "📤 **Send me a video file first:**"
            )

        except Exception as e:
            await message.reply_text(f"❌ Error: {str(e)}")

    @app.on_message(filters.command("mode2"))
    async def mode2_command(client, message: Message):
        """Handle /mode2 command"""
        try:
            user_id = message.from_user.id
            user_state = bot_instance.get_user_state(user_id)

            if user_state.get("processing"):
                await message.reply_text("⏳ **Please wait for current process to finish!**")
                return

            user_state["mode"] = "mode2"
            user_state["current_episode"] = 1

            await message.reply_text(
                "🤖 **Mode 2: Auto Rename from Metadata**\n\n"
                "**How it works:**\n"
                "1️⃣ Send me video files\n"
                "2️⃣ I'll extract metadata automatically\n"
                "3️⃣ Download, rename with template, and re-upload\n\n"
                f"📝 **Current Format:** `{bot_instance.settings.get('mode2_format')}`\n\n"
                "**Variables:**\n"
                "• `{name}` - Series name\n"
                "• `{episode}` - Episode number\n"
                "• `{resolution}` - Video resolution\n"
                "• `{ext}` - File extension\n\n"
                "📤 **Send video files to start:**"
            )

        except Exception as e:
            await message.reply_text(f"❌ Error: {str(e)}")

    @app.on_message(filters.command("status"))
    async def status_command(client, message: Message):
        """Handle /status command"""
        try:
            user_id = message.from_user.id
            user_state = bot_instance.get_user_state(user_id)

            status_text = (
                f"📊 **Bot Status (Google Colab)**\n\n"
                f"**User State:**\n"
                f"• Mode: {user_state.get('mode', 'None')}\n"
                f"• Processing: {'🟢 Yes' if user_state.get('processing') else '🔴 No'}\n"
                f"• Thumbnail: {'🖼️ Set' if user_state.get('thumbnail') else '❌ None'}\n"
                f"• Current Episode: {user_state.get('current_episode', 1)}\n\n"
                f"**Settings:**\n"
                f"• Mode2 Format: `{bot_instance.settings.get('mode2_format')}`\n"
                f"• Preserve Extension: {'✅' if bot_instance.settings.get('preserve_extension') else '❌'}\n\n"
                f"**Environment:** Google Colab\n"
                f"**Time:** {datetime.now().strftime('%H:%M:%S')}"
            )

            await message.reply_text(status_text)

        except Exception as e:
            await message.reply_text(f"❌ Error: {str(e)}")

    @app.on_message(filters.command("reset"))
    async def reset_command(client, message: Message):
        """Handle /reset command"""
        try:
            user_id = message.from_user.id

            if bot_instance.get_user_state(user_id).get("processing"):
                await message.reply_text("⏳ **Cannot reset while processing! Please wait for current task to finish.**")
                return

            bot_instance.reset_user_state(user_id)

            await message.reply_text(
                "🔄 **Bot State Reset**\n\n"
                "✅ **Cleared:**\n"
                "• Current mode\n"
                "• Thumbnail\n"
                "• Episode counter\n"
                "• All waiting states\n\n"
                "💡 **Use `/start` to see available commands**"
            )

        except Exception as e:
            await message.reply_text(f"❌ Error: {str(e)}")

    @app.on_message(filters.text & ~filters.regex(r"^/"))
    async def handle_text(client, message: Message):
        """Handle text messages"""
        try:
            user_id = message.from_user.id
            user_state = bot_instance.get_user_state(user_id)

            # Check if processing
            if user_state.get("processing"):
                await message.reply_text("⏳ **Please wait for current process to finish!**")
                return

            # Handle single name for Mode 1
            if user_state.get("mode") == "mode1" and user_state.get("temp_video"):
                new_name = message.text.strip()
                if not new_name:
                    await message.reply_text("❌ **Empty name!** Please send a valid filename")
                    return

                await process_mode1_simple(client, message, user_state, new_name, bot_instance)
                return

            # Default response
            await message.reply_text(
                "❓ **Unrecognized input**\n\n"
                "💡 **Try these commands:**\n"
                "• `/start` - Main menu\n"
                "• `/mode1` - Simple rename\n"
                "• `/mode2` - Auto rename\n"
                "• `/status` - Check status"
            )

        except Exception as e:
            await message.reply_text(f"❌ Error: {str(e)}")

    @app.on_message(filters.video | filters.document)
    async def handle_video(client, message: Message):
        """Handle video files - Download, rename, and re-upload"""
        try:
            user_id = message.from_user.id
            user_state = bot_instance.get_user_state(user_id)

            # Check if processing
            if user_state.get("processing"):
                await message.reply_text("⏳ **Please wait for current process to finish!**")
                return

            # Get file info with proper debugging
            file_id = None
            file_name = None
            file_size = 0

            if message.video:
                file_id = message.video.file_id
                file_name = message.video.file_name or f"video_{file_id[:10]}.mp4"
                file_size = message.video.file_size or 0
                print(f"DEBUG: Video file detected - name: {file_name}, size: {file_size} bytes")
            elif message.document and bot_instance.is_video_file(message.document.file_name or ""):
                file_id = message.document.file_id
                file_name = message.document.file_name
                file_size = message.document.file_size or 0
                print(f"DEBUG: Document video detected - name: {file_name}, size: {file_size} bytes")
            else:
                print("DEBUG: Not a video file or unsupported format")
                return

            # Ensure we have valid file size
            if file_size == 0:
                print("WARNING: File size is 0, this might cause progress issues")

            current_mode = user_state.get("mode")

            if not current_mode:
                await message.reply_text(
                    "⚠️ **No mode selected**\n\n"
                    "Please choose a mode first:\n"
                    "• `/mode1` - Simple rename\n"
                    "• `/mode2` - Auto rename"
                )
                return

            if current_mode == "mode1":
                # Simple mode - store video and ask for name
                if user_state.get("waiting_for_video"):
                    user_state["temp_video"] = {
                        "file_id": file_id,
                        "file_name": file_name,
                        "file_size": file_size
                    }
                    user_state["waiting_for_video"] = False

                    print(f"DEBUG: Stored video info - file_size: {file_size} bytes ({file_size/(1024*1024):.1f} MB)")

                    await message.reply_text(
                        f"📹 **Video Received!**\n\n"
                        f"📝 **File:** `{file_name}`\n"
                        f"📊 **Size:** {file_size / (1024*1024):.1f} MB\n\n"
                        f"💬 **Now send me the new filename:**"
                    )
                else:
                    await message.reply_text("❌ **Please use `/mode1` first to start the process**")

            elif current_mode == "mode2":
                await process_mode2_video(client, message, user_state, file_id, file_name, file_size, bot_instance)

        except Exception as e:
            logger.error(f"Error handling video: {e}")
            await message.reply_text(f"❌ Error processing video: {str(e)}")

async def process_mode1_simple(client, message: Message, user_state: dict, new_name: str, bot_instance):
    """Process Mode 1 simple rename"""
    try:
        user_state["processing"] = True

        video_info = user_state.get("temp_video")
        if not video_info:
            await message.reply_text("❌ **No video found!** Please start over with `/mode1`")
            user_state["processing"] = False
            return

        print(f"DEBUG: process_mode1_simple - video_info file_size: {video_info.get('file_size', 0)} bytes")

        # Add extension if needed
        if bot_instance.settings.get("preserve_extension", True):
            original_ext = Path(video_info["file_name"]).suffix
            if not new_name.endswith(original_ext):
                new_name += original_ext
        else:
            if not new_name.endswith('.mp4'):
                new_name += '.mp4'

        # Create progress message
        progress_msg = await message.reply_text(
            f"🔄 **Processing Simple Rename**\n\n"
            f"📹 **Original:** `{video_info['file_name']}`\n"
            f"📝 **New Name:** `{new_name}`\n\n"
            f"⏳ **Initializing...**"
        )

        try:
            download_path = os.path.join(DOWNLOAD_DIR, new_name)

            # Download with progress - ensure file_size is passed correctly
            file_size_bytes = video_info.get("file_size", 0)
            print(f"DEBUG: Calling download_with_progress with file_size: {file_size_bytes} bytes")

            await download_with_progress(
                client,
                video_info["file_id"],
                download_path,
                progress_msg,
                video_info["file_name"],
                file_size_bytes
            )

            # Upload with progress
            caption = new_name if bot_instance.settings.get("caption_mode") == "custom" else video_info["file_name"]

            await upload_with_progress(
                client,
                message.chat.id,
                download_path,
                progress_msg,
                new_name,
                f"📹 **{caption}**",
                user_state.get("thumbnail")
            )

            # Cleanup
            if os.path.exists(download_path):
                os.remove(download_path)

            await progress_msg.delete()

            await message.reply_text(
                f"✅ **Simple Rename Complete!**\n\n"
                f"📝 **Renamed to:** `{new_name}`\n\n"
                f"💡 **Use `/mode1` again for another video**"
            )

            # Reset state
            user_state["temp_video"] = None
            user_state["mode"] = None

        except Exception as e:
            await progress_msg.edit_text(f"❌ **Error:** {str(e)}")
            # Cleanup on error
            download_path = os.path.join(DOWNLOAD_DIR, new_name)
            if os.path.exists(download_path):
                os.remove(download_path)

        finally:
            user_state["processing"] = False

    except Exception as e:
        user_state["processing"] = False
        await message.reply_text(f"❌ Error in Mode 1: {str(e)}")

async def process_mode2_video(client, message: Message, user_state: dict, file_id: str, file_name: str, file_size: int, bot_instance):
    """Process Mode 2 video"""
    try:
        user_state["processing"] = True

        print(f"DEBUG: process_mode2_video - file_size: {file_size} bytes")

        # Extract metadata
        metadata = bot_instance.extract_metadata(file_name)
        episode_num = user_state.get("current_episode", 1)

        # Generate new filename
        new_filename = bot_instance.format_filename_mode2(metadata, episode_num)

        # Preserve extension if enabled
        if bot_instance.settings.get("preserve_extension", True):
            original_ext = Path(file_name).suffix
            if original_ext:
                new_filename = Path(new_filename).stem + original_ext

        # Create progress message
        progress_msg = await message.reply_text(
            f"🤖 **Mode 2: Auto Rename (Episode {episode_num})**\n\n"
            f"📹 **Original:** `{file_name}`\n"
            f"📝 **New Name:** `{new_filename}`\n"
            f"📊 **Size:** {file_size / (1024*1024):.1f} MB\n\n"
            f"**Extracted Metadata:**\n"
            f"• **Name:** {metadata.get('name', 'Unknown')}\n"
            f"• **Episode:** {metadata.get('episode', 'N/A')}\n"
            f"• **Resolution:** {metadata.get('resolution', 'N/A')}\n\n"
            f"⏳ **Initializing...**"
        )

        try:
            download_path = os.path.join(DOWNLOAD_DIR, new_filename)

            # Download with progress
            print(f"DEBUG: Calling download_with_progress with file_size: {file_size} bytes")

            await download_with_progress(
                client,
                file_id,
                download_path,
                progress_msg,
                file_name,
                file_size
            )

            # Upload with progress
            caption = new_filename if bot_instance.settings.get("caption_mode") == "custom" else file_name

            await upload_with_progress(
                client,
                message.chat.id,
                download_path,
                progress_msg,
                new_filename,
                f"📹 **{caption}**",
                user_state.get("thumbnail")
            )

            # Cleanup
            if os.path.exists(download_path):
                os.remove(download_path)

            await progress_msg.delete()

            # Increment episode counter
            user_state["current_episode"] = episode_num + 1

            await message.reply_text(
                f"✅ **Episode {episode_num} Complete!**\n\n"
                f"📝 **Renamed to:** `{new_filename}`\n"
                f"🔢 **Next Episode:** {episode_num + 1}\n\n"
                f"🎬 **Send next video or use `/reset` to restart**"
            )

        except Exception as e:
            await progress_msg.edit_text(f"❌ **Error:** {str(e)}")
            # Cleanup on error
            download_path = os.path.join(DOWNLOAD_DIR, new_filename)
            if os.path.exists(download_path):
                os.remove(download_path)

        finally:
            user_state["processing"] = False

    except Exception as e:
        user_state["processing"] = False
        await message.reply_text(f"❌ Error in Mode 2: {str(e)}")

# Export the functions for use in the main script
__all__ = ['create_bot_app', 'setup_handlers', 'VideoRenameBot']

Writing video_rename_bot.py


In [9]:

import asyncio
import os
import nest_asyncio
from video_rename_bot import create_bot_app, setup_handlers, VideoRenameBot

# Enable nested event loops for Colab
nest_asyncio.apply()

# Check if credentials are set
if API_ID == "YOUR_API_ID" or API_HASH == "YOUR_API_HASH" or BOT_TOKEN == "YOUR_BOT_TOKEN":
    print("❌ Please set your API_ID, API_HASH, and BOT_TOKEN in Cell 3!")
else:
    print("🎬 Video Rename Bot Starting on Google Colab...")
    print("📁 Downloads directory: /content/downloads")
    print("📋 Mode 1: Simple one-video rename")
    print("🤖 Mode 2: Auto rename from metadata")
    print("🚀 Bot is ready!")

    # Initialize bot
    bot_instance = VideoRenameBot()
    app = create_bot_app(API_ID, API_HASH, BOT_TOKEN)
    setup_handlers(app, bot_instance)

    # Define async function to run the bot
    async def run_bot():
        try:
            await app.start()
            print("✅ Bot started successfully!")
            print("💬 Send /start to your bot to begin")

            # Keep the bot running
            await asyncio.Event().wait()

        except KeyboardInterrupt:
            print("\n🛑 Bot stopped by user")
        except Exception as e:
            print(f"❌ Bot error: {e}")
        finally:
            try:
                await app.stop()
                print("🧹 Bot stopped gracefully")
            except:
                pass

            # Cleanup
            try:
                import shutil
                if os.path.exists("/content/downloads"):
                    shutil.rmtree("/content/downloads")
                    print("✅ Downloads directory cleaned")
            except Exception as cleanup_error:
                print(f"⚠️ Cleanup warning: {cleanup_error}")

    # Create a task to run the bot
    bot_task = asyncio.create_task(run_bot())

    print("\n🔄 Bot is now running in the background!")
    print("📱 Go to your Telegram bot and send /start")
    print("⚠️ To stop the bot, interrupt the kernel or run the stop cell")

🎬 Video Rename Bot Starting on Google Colab...
📁 Downloads directory: /content/downloads
📋 Mode 1: Simple one-video rename
🤖 Mode 2: Auto rename from metadata
🚀 Bot is ready!

🔄 Bot is now running in the background!
📱 Go to your Telegram bot and send /start
⚠️ To stop the bot, interrupt the kernel or run the stop cell


In [ ]:

# Run this cell to stop the bot
try:
    # Cancel the bot task
    if 'bot_task' in globals():
        bot_task.cancel()
        print("🛑 Bot task cancelled")

    # Stop the app if it exists
    if 'app' in globals():
        import asyncio
        async def stop_app():
            try:
                await app.stop()
                print("✅ Bot stopped successfully")
            except:
                print("ℹ️ Bot was already stopped")

        await stop_app()

    # Cleanup downloads
    import os
    import shutil
    if os.path.exists("/content/downloads"):
        shutil.rmtree("/content/downloads")
        print("🧹 Downloads directory cleaned")

except Exception as e:
    print(f"⚠️ Stop process completed with warnings: {e}")

print("✅ Bot shutdown complete")